In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use("fivethirtyeight")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
100 * data.isna().sum() / data.shape[0]

In [ ]:
data.isna().sum()

In [ ]:
data.keyword[55:232]

In [ ]:
data["keyword"].fillna("oov", inplace=True)
data["location"].fillna("unknown", inplace=True)

In [ ]:
data.head()

In [ ]:
data.drop("id", axis=1, inplace=True)

In [ ]:
x, y = data.drop("target", axis=1), data["target"]

In [ ]:
def transform_to_one_column(r):
    return r['keyword'] + ' ' + r["location"] + ' ' + r["text"]

In [ ]:
x = x.apply(transform_to_one_column, axis="columns")

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import string
punct = string.punctuation

In [ ]:
def process(s):
    for p in punct:
        s = s.replace(p, '')
    s = s.lower()
    s = word_tokenize(s)
    s = " ".join(s)
    s = lemmatizer.lemmatize(s)
    return s

In [ ]:
x = x.map(process)

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(x=y, palette="cool")
plt.title("\nDistribution of the Target Feature\n\n", fontsize=35)
plt.show()

In [ ]:
def wordcount(r):
    return len(r.split())

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(x=x.apply(wordcount), color="mediumspringgreen")
plt.title("\nn° Words per Document\n\n", fontsize=35)
plt.show()

In [ ]:
mydata = pd.concat([x,y], axis=1).rename({0:"text"}, axis=1)

In [ ]:
mydata.head()

In [ ]:
mydata.to_csv("mydata.csv", index=False)

In [ ]:
#Huggingface
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
mydata

In [ ]:
mydata = load_dataset("csv", data_files="./mydata.csv")["train"]

In [ ]:
mydata

In [ ]:
mydata = mydata.train_test_split(test_size=0.1)

In [ ]:
mydata

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
tokenizer.model_max_length

In [ ]:
def tokenize(e):
    return tokenizer(e["text"], truncation= True)

In [ ]:
#mapping data through HF tokenizer
md = mydata.map(tokenize, batched= True)

In [ ]:
md

In [ ]:
md.column_names

In [ ]:
traindata = md["train"]
testdata = md["test"]

In [ ]:
traindata = traindata.remove_columns(["text"]).rename_column('target', "labels").with_format("torch")
testdata = testdata.remove_columns(["text"]).rename_column('target', "labels").with_format("torch")

In [ ]:
traindata #after dropping ["text"]

In [ ]:
traindata.select(range(10))

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [ ]:
datacol = DataCollatorWithPadding(tokenizer)
trainloader = DataLoader(traindata, batch_size=32, shuffle=True, collate_fn=datacol)
testloader = DataLoader(testdata, batch_size=32, collate_fn=datacol)

In [ ]:
traindata

In [ ]:
#Modle part
!pip install accelerate

In [ ]:
from accelerate import Accelerator   #for distributed settings training

In [ ]:
accelerator = Accelerator()

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
checkpoint="roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import AdamW
optim = AdamW(model.parameters(), lr=5e-5)   #5e-5, 6e-6 , 3e-4, 1e-06
#why AdamW over Adam
#https://towardsdatascience.com/why-adamw-matters-736223f31b5d

In [ ]:
from tqdm.auto import tqdm
import torch

In [ ]:
device = accelerator.device
model=model.to(device)
print(device)

In [ ]:
from datasets import load_metric
f1 = load_metric("f1")
acc = load_metric("accuracy")

In [ ]:
from copy import deepcopy

In [ ]:
model, optimizer, trainloader = accelerator.prepare(model, optim, trainloader)
testloader = accelerator.prepare(testloader)

In [ ]:
def validate(model):
    for batch in testloader:
        #batch = {k:v.cuda() for k,v in batch.items()}
        outputs = model(**batch)
        predictions=torch.argmax(outputs.logits, dim=-1)
        #f1.add_batch(predictions=predictions, references=batch["labels"])
        f1.add_batch(predictions=accelerator.gather(predictions), references=accelerator.gather(batch["labels"]))
        #acc.add_batch(predictions=predictions, references=batch["labels"])
        acc.add_batch(predictions=accelerator.gather(predictions), references=accelerator.gather(batch["labels"]))
    acc_res = acc.compute()["accuracy"]
    print(f"Validation Accuracy: {acc_res:.2f}")
    f_res = f1.compute()["f1"]
    print(f"Validation F1-score: {f_res:.2f}")
    return acc_res

In [ ]:
nsteps= 214
nepoch= 10
best_val_acc = 0
for epoch in range(nepoch):
    model.train()
    print(f"epoch n°{epoch+1}:")
    av_epoch_loss=0
    progress_bar = tqdm(range(nsteps))
    for batch in trainloader:
        #batch = {k:v.cuda() for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        av_epoch_loss += loss
        #loss.backward()
        accelerator.backward(loss)
        optim.step()
        optim.zero_grad()
        predictions=torch.argmax(outputs.logits, dim=-1)
        f1.add_batch(predictions=predictions, references=batch["labels"])
        acc.add_batch(predictions=predictions, references=batch["labels"])
        progress_bar.update(1)
    av_epoch_loss /= nsteps
    print(f"Training Loss: {av_epoch_loss: .2f}")
    acc_res = acc.compute()["accuracy"]
    print(f"Training Accuracy: {acc_res:.2f}")
    f_res = f1.compute()["f1"]
    print(f"Training F1-score: {f_res:.2f}")
    model.eval()
    val_acc = validate(model)
    if val_acc > best_val_acc:
        print("Achieved best validation accuracy so far. Saving model.")
        best_val_acc = val_acc
        best_model_state = deepcopy(model.state_dict())
    print("\n\n")

In [ ]:
model.load_state_dict(best_model_state)

In [ ]:
sub_data = pd.read_csv("../input/nlp-getting-started/test.csv")
sub_data["keyword"].fillna("oov", inplace=True)
sub_data["location"].fillna("unknown", inplace= True)
sub_data.drop("id", axis=1, inplace= True)
sub_data = sub_data.apply(transform_to_one_column, axis="columns")
sub_data = sub_data.map(process).to_frame().rename({0:"text"}, axis=1)
sub_data = Dataset.from_pandas(sub_data)
sub_data = sub_data.map(tokenize, batched= True)
sub_data = sub_data.remove_columns(["text"]).with_format("torch")

In [ ]:
sub_loader = DataLoader(sub_data, batch_size=32, collate_fn=datacol)
preds = torch.Tensor().cuda()
for batch in sub_loader:
        batch = {k:v.cuda() for k,v in batch.items()}
        outputs = model(**batch)
        preds=torch.cat((preds,torch.argmax(outputs.logits, dim=-1)))
preds = pd.Series(preds.cpu()).astype("int64")

In [ ]:
#submission columns should be id, target
id = pd.read_csv("../input/nlp-getting-started/test.csv")["id"]
submission = pd.DataFrame({"id":id, "target":preds})
submission.to_csv("submission.csv", index= False)

In [ ]:
#checking the submission file for same format
import pandas as pd
sample_sub = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_sub.head()
